# SKT-koBERT

https://github.com/SKTBrain/KoBERT

In [1]:
hanspell_path = '/home/hyejeongeun/Todays_Chatbot/Data/QnA/QnA_preprocessing/QnA_hanspell/'
mecab_path = '/home/hyejeongeun/Todays_Chatbot/Data/QnA/QnA_preprocessing/QnA_mecab/'

## Install & Library

In [3]:
!nvidia-smi

Mon May 17 10:58:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:40:00.0 Off |                  N/A |
| 32%   36C    P8     5W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:60:00.0 Off |                  N/A |
| 29%   

In [1]:
# !git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 162 (delta 1), reused 2 (delta 0), pack-reused 155
Receiving objects: 100% (162/162), 184.74 KiB | 534.00 KiB/s, done.
Resolving deltas: 100% (83/83), done.


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})

# CPU or GPU
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
device

'cuda:1'

## Model & Tokenizer

In [3]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [4]:
# 내장된 tokenizer와 sentencepiece 함수를 쓰는 듯
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


---

# Que [SEP] Ans

## Load Data & Setting

In [8]:
# 4. mecab + 카테고리 5개(기타 미포함)

In [5]:
# 데이터 불러오기
dataset_train = nlp.data.TSVDataset(mecab_path+'qna_mecab_drop_train.txt')
dataset_test = nlp.data.TSVDataset(mecab_path+'qna_mecab_drop_test.txt')

In [6]:
len(dataset_train), len(dataset_test)

(73337, 18335)

In [7]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [8]:
## Setting parameters
max_len = 210 #수정
batch_size = 16 # 64 #수정
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [9]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [10]:
len(data_train), len(data_test)

(73337, 18335)

In [11]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## BERT Classifier

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [14]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [15]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [16]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        del out # 데이터 반환
        torch.cuda.empty_cache() # GPU 캐시 삭제
        
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        del out # 데이터 반환
        torch.cuda.empty_cache() # GPU 캐시 삭제
        
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
    # add: model save code
    model_path = "/home/hyejeongeun/Todays_Chatbot/Model/Model/epoch_"+str(e+1)+"_qna_sep_model_4.pt"
    torch.save(model.state_dict(), model_path)
    print("Save model")

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4584 [00:00<?, ?it/s]

epoch 1 train acc 0.8124621267209617


/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1146 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8572171320535195
Save model


  0%|          | 0/4584 [00:00<?, ?it/s]

epoch 5 train acc 0.9086663152026372


  0%|          | 0/1146 [00:00<?, ?it/s]

epoch 5 test acc 0.8556355439208841
Save model


----